In [3]:
from huggingface_hub import login
import os

hf_token = os.getenv("HF_token")  
login(token=hf_token)

In [2]:
from datasets import load_dataset

ds = load_dataset("sionic-ai/hme100k")


KeyboardInterrupt: 

In [3]:
import torch
from datasets import load_dataset
from transformers import (
    NougatProcessor,
    VisionEncoderDecoderModel,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
from PIL import Image
import re

In [4]:
from datasets import load_dataset
from transformers import NougatProcessor  # ⚠️ не забудьте импорт!

# Инициализация processor
processor = NougatProcessor.from_pretrained("facebook/nougat-base")

# Загрузка датасета
dataset = load_dataset("sionic-ai/hme100k", cache_dir="/tmp/hme100k")

def preprocess_batch(batch):
    images = [img.convert("RGB") for img in batch["image"]]
    pixel_values = processor(images, return_tensors="pt").pixel_values

    latex_strs = [conv[1]["content"] for conv in batch["conversations"]]
    labels = processor.tokenizer(
        latex_strs,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512,
    ).input_ids

    return {
        "pixel_values": pixel_values,
        "labels": labels
    }

# Теперь processor доступен внутри preprocess_batch
dataset = dataset.map(
    preprocess_batch,
    batched=True,
    batch_size=32,
    remove_columns=dataset["train"].column_names,
    cache_file_names={
        "train": "/tmp/hme100k/train.processed.arrow",
        "validation": "/tmp/hme100k/val.processed.arrow"
    },
    num_proc=1
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

ValueError: Make sure to set the decoder_start_token_id attribute of the model's configuration.